<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day7_Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q -U google-generativeai tavily-python pypdf

In [20]:
import os
import io
import json
import datetime
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
from IPython.display import display, Markdown, clear_output

# --- CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set keys in Colab Secrets.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest')
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("Setup complete.")

Setup complete.


In [21]:
# --- MEMORY SYSTEM (The New Part) ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:50] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

# --- HELPER FUNCTIONS ---
def get_pdf_text(uploaded_file):
    pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

def generate_plan(source_text, mode="topic"):
    if mode == "topic":
        prompt = f"Break down this topic into 3 search queries: {source_text}. Return ONLY 3 lines."
    else:
        prompt = f"Identify 3 validation questions for this paper text: {source_text[:10000]}. Return ONLY 3 lines."

    response = model.generate_content(prompt)
    return [q.strip() for q in response.text.strip().split('\n') if q.strip()]

def perform_research(questions):
    results = []
    print(f"🔎 Searching...")
    for q in questions:
        try:
            res = tavily.search(query=q, search_depth="basic", max_results=1)
            results.append(f"Q: {q}\nA: {res['results'][0]['content']}")
        except:
            pass
    return "\n".join(results)

def synthesize_report(source_text, research_data, mode):
    prompt = f"Summarize user input '{source_text[:5000]}' combined with search data '{research_data}' into a Markdown report."
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# --- MAIN APPLICATION LOOP ---

def main():
    while True:
        print("\n" + "="*10 + " AI RESEARCH ASSISTANT " + "="*10)
        print("1. New Research (Topic)")
        print("2. New Research (PDF)")
        print("3. 📜 View Past Inputs & Reports")
        print("4. Exit")

        choice = input("\nSelect Option (1-4): ")

        if choice == '4':
            print("Goodbye! 👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # --- OPTIONS 1 & 2: NEW RESEARCH ---
        source_text = ""
        mode = ""

        if choice == '1':
            mode = "topic"
            source_text = input("Enter Topic: ")
        elif choice == '2':
            mode = "pdf"
            print("Upload PDF:")
            uploaded = files.upload()
            if uploaded:
                fn = next(iter(uploaded))
                source_text = get_pdf_text(uploaded[fn])
        else:
            print("Invalid choice.")
            continue

        if source_text:
            # Execute Agent Logic
            plan = generate_plan(source_text, mode)
            data = perform_research(plan)
            report = synthesize_report(source_text, data, mode)

            # Output
            print("\n🎯 FINAL REPORT:")
            display(Markdown(report))

            # Save to Memory
            memory.save_entry(source_text, mode, report)

if __name__ == "__main__":
    main()


========== AI RESEARCH ASSISTANT ==========
1. New Research (Topic)
2. New Research (PDF)
3. 📜 View Past Inputs & Reports
4. Exit

Select Option (1-4): 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-26 13:23   | pdf      | Reference Quadrupole Moments of Transition Element...
2    | 2025-11-26 13:24   | pdf      | Reference Quadrupole Moments of Transition Element...
3    | 2025-11-26 13:50   | pdf      | Reference Quadrupole Moments of Transition Element...

Enter ID to view full report (or Press Enter to go back): 1

🔁 RECALLING REPORT FOR: Reference Quadrupole Moments of Transition Element...
--------------------------------------------------


Of course. Here is a summary of the provided text and an analysis of its core claims, formatted as a Markdown report.

---

### **Report on "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms"**

#### **I. Summary of the Paper**

This paper by S. Rathi et al. proposes a new experimental method to significantly improve the accuracy of measuring the absolute electric quadrupole moment (Q) of light transition elements (atomic numbers Z=23 to Z=30). The quadrupole moment is a fundamental property that describes the deviation of an atomic nucleus from a perfect sphere and is crucial for understanding nuclear structure.

**The Problem:**
*   Current methods for determining Q rely on measuring a property called the hyperfine coupling constant (B) and then dividing it by a calculated electric field gradient (EFG, or Vzz).
*   For transition elements, which have complex open-shell electron structures (partially filled d-orbitals), calculating the EFG accurately is extremely difficult. This calculation is the dominant source of uncertainty.
*   The accuracy of this single "reference" quadrupole moment (Q_ref) for one isotope limits the accuracy for an entire chain of related isotopes, as their moments are measured relative to it.

**The Proposed Solution:**
*   The authors propose using **muonic atoms**, where an electron is replaced by a much heavier muon. In such an atom, the muon orbits very close to the nucleus, making the system much simpler and the EFG almost entirely determined by the muon's position. This bypasses the need for complex and uncertain many-electron calculations.
*   They will perform high-precision spectroscopy on the energy difference between the 2s and 2p muonic states (the "Lamb shift").
*   This measurement requires specialized detectors, as the signals are weak and overlapping. The authors propose using **cryogenic microcalorimeters**, which offer the necessary high energy resolution and efficiency.

**Expected Outcome:**
*   Based on simulations, the authors claim their method can **reduce the uncertainty in the reference quadrupole moments by up to an order of magnitude** (i.e., a factor of 10).
*   This high-precision measurement could be achieved within a single day of experiment time.
*   These improved reference values will provide a stronger foundation for nuclear structure studies and serve as critical benchmarks for advancing quantum chemistry theories.

---

#### **II. Validation Questions Probing Core Claims**

Based on the text, the paper makes a **causal claim**: that implementing their proposed method (muonic spectroscopy with microcalorimeters) will *cause* a significant reduction in the uncertainty of reference quadrupole moments. To evaluate this claim, one must assess its **construct, internal, statistical, and external validity**. The following three questions probe these validities.

**Question 1 (Probing Internal & Construct Validity):**

> The paper's core assumption is that moving to a muonic system effectively eliminates the main source of theoretical error by bypassing complex electronic EFG calculations. However, the method still relies on "state-of-the-art theoretical calculations" for the muonic system itself. How sensitive is the final extracted quadrupole moment to these *remaining* theoretical corrections (e.g., vacuum polarization, nuclear polarization, self-energy)? Could these currently secondary uncertainties become the new limiting factor, preventing the full "order of magnitude" improvement claimed?

*   **Relevance:** This question tests the **internal validity** of the core claim. It asks if an unstated or minimized factor (theoretical errors in the muonic model) could be an alternative explanation for why the claimed precision might not be reached. It also probes **construct validity** by asking if the new method truly isolates the nuclear property (Q) as cleanly as suggested.

**Question 2 (Probing Statistical & External Validity):**

> The feasibility of the method is demonstrated via "extensive calculations and realistic simulations," not an actual experiment. How comprehensively do these simulations account for practical experimental noise and systematic effects, such as detector background, muon beam purity, target impurities, and Doppler broadening? A significant discrepancy between the idealized simulation and real-world conditions could drastically alter the required measurement time and the final achievable precision.

*   **Relevance:** This question challenges the **statistical validity** of the conclusions drawn from the simulations. It also directly addresses **external validity**—the ability to generalize the simulated results to a real-world laboratory setting. The claim of achieving the result in "a day of measurement" is critically dependent on the realism of these simulations.

**Question 3 (Probing External Validity):**

> The method is specifically proposed for light transition elements (23 ≤ Z ≤ 30). What are the fundamental physical or technical limitations that would prevent this technique from being applied to other regions of the nuclear chart, such as heavier elements or nuclei with very short lifetimes? For instance, do rapidly increasing muonic x-ray energies or more complex nuclear structure effects in heavier nuclei pose insurmountable challenges for the proposed cryogenic microcalorimeter setup?

*   **Relevance:** This question probes the **external validity** or generalizability of the proposed method. Understanding the boundaries and limitations of the technique is crucial for assessing its overall impact on the field of nuclear physics beyond the specific range of elements discussed.


Press Enter to continue...

========== AI RESEARCH ASSISTANT ==========
1. New Research (Topic)
2. New Research (PDF)
3. 📜 View Past Inputs & Reports
4. Exit
